In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

: 

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/Trimester02/Dataset/text.csv')
df

,Name,Position,Duration,jump
0,A,0,4571,1
1,B,2619,7191,1
2,C,7925,7691,0
3,D,13697,7174,0
4,E,18886,8408,1
5,F,26360,3287,1
6,G,27744,8458,0
7,H,34418,7374,0


In [ ]:
def extract_video_chunk(video_path, start_duration, end_duration, output_folder):
    # Check if output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the video
    cap = cv2.VideoCapture(video_path)

    # Check if the video is successfully loaded
    if not cap.isOpened():
        print("Error: Unable to open the video file.")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_duration = total_frames / fps * 1000  # Total duration in milliseconds

    # Convert start and end duration to frames
    start_frame = int(start_duration * fps / 1000)
    end_frame = int(end_duration * fps / 1000)

    # Check if the provided start and end durations are within the video duration
    if start_duration < 0 or start_duration > total_duration or end_duration < 0 or end_duration > total_duration:
        print("Error: Invalid start or end duration.")
        return

    # Set the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Read and save the video chunk
    frames = []
    for frame_num in range(start_frame, end_frame):
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break

    # Save the chunk as a video
    if frames:
        output_video_path = os.path.join(output_folder, f"video_chunk_{start_duration}_{end_duration}.mp4")
        out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frames[0].shape[1], frames[0].shape[0]))
        for frame in frames:
            out.write(frame)
        out.release()

    # Release the video capture
    cap.release()


In [ ]:
video_path = "/content/drive/MyDrive/Trimester02/Dataset/Raw_Videos/Scott Brash - Hello Jefferson - Aachen - 1.60m - 03.07.2022 Jumpoff.mp4"
start_duration = 0
end_duration = 0
output_folder = "/content/drive/MyDrive/Trimester02/Dataset/Chunks"

In [ ]:
positions = df['Position']

for i in range(1, len(positions)):
    start_duration = positions[i - 1]
    end_duration = positions[i]
    print("chunk:",start_duration,"-",end_duration)
    extract_video_chunk(video_path, start_duration, end_duration, output_folder)

In [ ]:
positions = df['Position']

# Iterate over the positions list and calculate differences
for i in range(len(positions)):
    if i == 0:
        start_duration = positions[i]
        end_duration = positions[i] + (positions[i + 1] - positions[i]) + 1000  # Add 1 second (1000 milliseconds) after the first chunk
    elif i == len(positions) - 1:
        start_duration = positions[i - 1] - 1000  # Subtract 1 second (1000 milliseconds) before the last chunk
        end_duration = positions[i]
    else:
        start_duration = positions[i - 1] - 1000  # Subtract 1 second (1000 milliseconds) for the start duration
        end_duration = positions[i] + 1000  # Add 1 second (1000 milliseconds) for the end duration

    print("Chunk:", start_duration, "-", end_duration)
    extract_video_chunk(video_path, start_duration, end_duration, output_folder)


Chunk: 0 - 3619
Chunk: -1000 - 3619
Error: Invalid start or end duration.
Chunk: 1619 - 8925
Chunk: 6925 - 14697
Chunk: 12697 - 19886
Chunk: 17886 - 27360
Chunk: 25360 - 28744
Chunk: 26744 - 34418


In [29]:
duration = df['Duration']

for dur in duration:
  print(dur)

for index, row in df.iterrows():
    start_duration = row['Position']
    duration = row['Duration']
    end_duration = start_duration + duration
    extract_video_chunk(video_path, start_duration+850, end_duration, output_folder)


4571
7191
7691
7174
8408
3287
8458
7374
